# Fullstack GPT: #5.0 ~ #5.8

- [x] Implement an LCEL chain with a memory that uses one of the memory classes we learned about.
- [x] The chain should take the title of a movie and reply with three emojis that represent the movie. (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").
- [x] Provide examples to the chain using FewShotPromptTemplate or FewShotChatMessagePromptTemplate to make sure it always replies with three emojis.
- [x] To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.

In [1]:
examples = [
    {
        "title": "Top Gun", 
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "title": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝"
    },
]

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def invoke_chain(title):
    result = chain.invoke({"title": title})
    memory.save_context({"input": title}, {"output": result.content})

llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.3,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
memory = ConversationBufferMemory(return_messages=True)

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{title}"),
    ("ai", "{answer}")
])

fewshot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "take the title of a movie and reply with three emojis that represent the movie."),
    MessagesPlaceholder(variable_name="history"),
    fewshot_prompt,
    ("human", "{title}")
])

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm
invoke_chain("Chocolate")
invoke_chain("Sister Act")

🍫❤️🍭🎤👩‍🎤⛪

In [4]:
chain.invoke({"title": "What is the movie i asked about first."})

The movie you asked about first is "Chocolate."

AIMessageChunk(content='The movie you asked about first is "Chocolate."')